In [ ]:
from scipy.integrate import odeint
from scipy.optimize import minimize

import matplotlib.pyplot as plot
import pandas as pd
import numpy as np
plot.style.use('seaborn')

experiment = 'test_calibration'

In [ ]:
def electric_heater(T, t, parameters, u):
    TL, Tr = T
    #### Parameters
    # U = global heat transfer coefficient
    # U_loss = heat losses to air
    # A = area of heat Transfer
    # T_amb = ambient temperature
    # L_mass = mass of the process liquid
    # gain = gain
    
    U_loss = parameters [0] # W / m^2 s
    U = parameters[1] # W / m^2 s
    gain = 1
    T_amb = 25
    L_mass = 200
    A = 20 # m^2
    R_mass = 50 # grams
    Cp = 4.186 # J / g K
    Cpr = 1 # J / g K
    
    Qt = A * U * (Tr - TL)
    Q_loss = A * U_loss * (Tr - T_amb)
    
    dTLdt =  Qt / (L_mass * Cp)
    dTrdt =  (gain * u - (Qt + Q_loss)) / (R_mass * Cpr)
    
    return [dTLdt, dTrdt]


def simulate(parameters):
    initial_conditions = data.loc[0, ["T_heater", "T_liquid"]].to_list()
    simulation_results = pd.DataFrame(columns=["Time", "u", "T_heater", "T_liquid"])
    simulation_results.loc[0, "Time"] = data.loc[0, "Time"]
    simulation_results.loc[0, "T_heater"] = initial_conditions[0]
    simulation_results.loc[0, "T_liquid"] = initial_conditions[1]
    simulation_results.loc[0, "u"] = data.loc[0, "PWM"]

    N = len(data)
    time = data.loc[0:, "Time"]
    u = data.loc[0:, "PWM"]

    for i in range(1, N):
        time_interval = [time[i - 1], time[i]]
        solution = odeint(
            electric_heater, initial_conditions, time_interval, args=(parameters, u[i - 0])
        )

        simulation_results.loc[i, "Time"] = time[i]
        simulation_results.loc[i, "T_heater"] = solution[-1, 0]
        simulation_results.loc[i, "T_liquid"] = solution[-1, 1]
        simulation_results.loc[i, "u"] = u[i]
        initial_conditions = [solution[-1, 0], solution[-1, 1]]

    return simulation_results


def squared_error(simulation_results):
    y = data.loc[:, ["T_heater", "T_liquid"]].to_numpy()
    yhat = simulation_results.loc[:, ["T_heater", "T_liquid"]].to_numpy()
    penalty = np.diag(0.8 * np.ones(len(data)))

    return np.dot(np.dot((y - yhat).transpose(), penalty), (y - yhat))


def model_deviation(simulation_results):
    yhat = simulation_results.loc[:, ["T_heater", "T_liquid"]].to_numpy()
    yhat_old = old_results.loc[:, ["T_heater", "T_liquid"]].to_numpy()
    penalty = np.diag(0.1 * np.ones(len(data)))

    return np.dot(np.dot((yhat - yhat_old).transpose(), penalty), (yhat - yhat_old))


def parameter_movement(parameters):
    p = np.array(parameters)
    p_old = parameters_df.iloc[-1, :].to_numpy()
    penalty = np.diag(0.1 * np.ones(len(p_old)))
    delta_p = (p - p_old)

    return np.dot(np.dot(delta_p.transpose(), penalty), delta_p)


def objective(parameters):
    simulation_results = simulate(parameters)
    objective = squared_error(simulation_results) + model_deviation(simulation_results) + parameter_movement(parameters)

    return objective / len(data)

In [ ]:
parameters_df = pd.DataFrame(columns=["U_loss", "U"])
parameters_df.loc[0, "U_loss"] = 1e-5   # Heat losses to air
parameters_df.loc[0, "U"] = 1e-4   # Heat transfer from heater to liquid
iter = 0

In [ ]:
%time
iter = iter + 1

# Load Data
data = pd.read_csv(experiment + '/data.csv')
data.loc[:, "Time"] = data.loc[:, "Time"] * 3600
data = data.reset_index().drop(columns=['index'])

# Simulate model with parameters
old_parameters = parameters_df.loc[iter - 1, :].to_list()
old_results = simulate(old_parameters)
new_parameters = minimize(objective, [1, 10], method='Nelder-Mead', tol=100)
parameters_df.loc[iter, :] = new_parameters.x

In [ ]:
plot.plot(data.loc[:, "Time"], data.loc[:, "T_heater"], '*', old_results.loc[:, "Time"], old_results.loc[:, "T_heater"])
plot.plot(data.loc[:, "Time"], data.loc[:, "T_liquid"], '*', old_results.loc[:, "Time"], old_results.loc[:, "T_liquid"])
plot.show()